In [1]:
#!/usr/bin/env python3
"""
Patient Timeline Creator
Creates a comprehensive timeline for patients by combining data from multiple tables.
"""

import pandas as pd
import duckdb as db
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:

def get_database_connection():
    """Create and return a connection to the clinisys_all database"""
    path_to_db = '../../database/huntington_data_lake.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def extract_timeline_data_for_patient(conn, test_prontuario):
    """Extract timeline data for a specific patient"""
    query = f"""
        SELECT * FROM gold.all_patients_timeline
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_finops_data(conn, test_prontuario):
    """Get finops data for a specific patient"""
    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_mescladas(conn, test_prontuario):
    """Get mescladas data for a specific patient"""
    query = f"""
        SELECT * FROM silver.mesclada_vendas
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def main(test_prontuario):
    """Main function to create patient timeline"""
    
    # Test with a single patient first
    # test_prontuario = 175583
    
    print(f"Creating timeline for patient: {test_prontuario}")
    
    # Get database connection
    conn = get_database_connection()
    
    try:
        # Extract timeline data
        timeline_df = extract_timeline_data_for_patient(conn, test_prontuario)
       
        finops_df = get_finops_data(conn, test_prontuario)

        mescladas_df = get_mescladas(conn, test_prontuario)
        return timeline_df, finops_df, mescladas_df
        
    finally:
        conn.close()

In [3]:
if __name__ == "__main__":

    ## casos para testes iniciais

    # patient_id = 825_890 # Claudia
    # patient_id = 175_583 # Renata
    # patient_id = 220_783 # caso cabelud0:: - 26 tentativas
    # patient_id = 182_925 # caso completo
    # patient_id = 162_173 # caso bonito: 5 ciclos - 5 pagamentos

    ## resolvidos
    # patient_id = 641_950 # 11 cycles without payment
    # patient_id = 150_491 # ovo recepção
    # patient_id = 155_282 # medico externo
    # patient_id = 738_349 # (marido 738_350 não aparece no clinisys) -- sem pagamento
    # patient_id = 839574 # utero de substituicao (815299)
    # patient_id = 842663 # doadora
    # patient_id = 678_207 # numero cliente duplicado no diario_vendas (78207) - BH
    # patient_id = 831_609 # pagamento no ID errado do paciente, mas certo no marido (CLiente = 833_788)
    # patient_id = 662_298 # paciente 50347 em mescladas, cliente 62298 no diario - BH
    
    # patient_id = 162733 # Beshaatova - CLIENTE/PACIENTE 64645 - Filipe
    # patient_id = 832_556 # utero de substituição - pago por 831_609 - Filipe
    # patient_id = 881_426 # utero de substituição - 126881 - Filipe

    ## abertos

    # patient_id = 880_484 # erro de migração de cadastro - pago  cliente 55623
    # patient_id = 867_007 # pago - nomes diferentes no clinisys e mescladas (erro de cadastro no Totvs)
    # patient_id = 875_610 # autorizado pela gerencia, sem custo
    # patient_id = 178125 # pago em 2021, primeira transferencia em 2025

    ## sem pagamento ou sem faturamento
    # patient_id = 166_697 # FOT R$ 4695 em 11/06/2025 + biópisia
    # patient_id = 172_179 # funcionaria 
    # patient_id = 885_517  # Matheus Roque - pago na Mater Prime
    # patient_id = 879_641 # Matheus Roque - pago na Mater Prime
    # patient_id = 887_035 # relato que está pago no orçamento, mas não lançado e não achamos NF
    # patient_id = 873_039 # não pago - FET
    

    ## depois da mudança no código
    patient_id = 655_506
    patient_id = 749579 



    timeline_df, finops_df, mescladas_df = main(patient_id)


Creating timeline for patient: 749579
Connected to database: ../../database/huntington_data_lake.duckdb
Database file exists: True


In [4]:
finops_df

,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,treatment_paid_count,treatment_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,prontuario_genitores,medico_nome,flag_is_donor,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments,timeline_unidade
0,749579.0,2,1,3,8.0,65001.99,2024-01-16,2024-08-29,2023-08-31,2024-08-21,749579.0,Livia Munhoz,0,0,0,0,1,6. HTT Brasília


In [5]:
timeline_df

,prontuario,event_id,event_date,reference,reference_value,tentativa,unidade,resultado_tratamento,flag_date_estimated,additional_info
0,749579.0,1306952,2024-12-19,extrato_atendimentos,1º US Obstétrico: Confirmação,None,6. HTT Brasília,None,False,{}
1,749579.0,16283,2024-11-11,descongelamentos_embrioes,2877/24,None,None,None,False,"{""CodDescongelamento"": ""2425/24"", ""Unidade"": ""5""}"
2,749579.0,1282462,2024-11-11,extrato_atendimentos,FET: Ciclo Próprio,None,6. HTT Brasília,None,False,{}
3,749579.0,30408,2024-11-04,tratamentos,Ciclo de Congelados,8,6. HTT Brasília,None,True,{}
4,749579.0,1279131,2024-11-04,extrato_atendimentos,2º US Ciclo,None,6. HTT Brasília,None,False,{}
5,749579.0,1274250,2024-11-01,extrato_atendimentos,Avaliação clínica (sem cobrança),None,6. HTT Brasília,None,False,{}
6,749579.0,1277545,2024-10-30,extrato_atendimentos,1° US Ciclo,None,6. HTT Brasília,None,False,{}
7,749579.0,1259916,2024-09-27,extrato_atendimentos,Consulta de Acompanhamento Psicológico ***,None,6. HTT Brasília,None,False,{}
8,749579.0,1247685,2024-09-27,extrato_atendimentos,Avaliação clínica (sem cobrança),None,6. HTT Brasília,None,False,{}
9,749579.0,1252251,2024-09-19,extrato_atendimentos,Consulta de Acompanhamento Psicológico ***,None,6. HTT Brasília,None,False,{}


In [6]:
mescladas_df.sort_values(by='DT Emissao', ascending=False)

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-09-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,27178,3,26431,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,24444,20251013_151805,Mescladas2 (2).xlsx
83,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-08-18,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,26043,3,25292,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,479992,20251013_151805,Mescladas2 (2).xlsx
82,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-07-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,24766,3,24023,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,478529,20251013_151805,Mescladas2 (2).xlsx
81,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-06-16,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,23613,3,22885,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,477259,20251013_151805,Mescladas2 (2).xlsx
80,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-05-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,22393,3,21670,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,475878,20251013_151805,Mescladas2 (2).xlsx
79,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-04-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,21273,3,20550,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,474650,20251013_151805,Mescladas2 (2).xlsx
78,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-03-17,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,20212,3,19492,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,473476,20251013_151805,Mescladas2 (2).xlsx
77,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-02-17,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,19207,3,18488,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,472352,20251013_151805,Mescladas2 (2).xlsx
76,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-01-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,18015,3,17306,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,470978,20251013_151805,Mescladas2 (2).xlsx
75,749582,GILBERTO RIBEI,817292,DANIELA ALVES AMORIM PELUCIO,749579,2024-12-19,UTROGESTAN 200MG C 14 CAPSULAS,6.0,474.00,Estimulação,30101,510,17310,3,16603,109,LIVIA OLIVEIRA MUNHOZ,879432,,0000000004632DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,11310100005,,,Estim